# Análise Estatística de Dados e Informações
**PPCA/UNB**

---

**Tarefa 03**  
**Professor**: João Gabriel de Moraes Sousa  
**Aluna**: Andreia Queiroz Correia Dummar  
**Matrícula**: 241134680  
**Data da Entrega**: 08/12/2024  
**Github**: https://github.com/aqcorreia/AEDI/tree/bb68c12f3e69d6cafe3fd7d6864e38b4c0721182/Tarefa03

---

**Aplicação**: Análise estatística com ANOVA

**Dados**: https://www.kaggle.com/prevek18/ames-housing-dataset
- Análise considerou as características: No de quartos, bairros e número de garagens


## Bibliotecas

In [ ]:
# Importação de bibliotecas essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos estatísticos e testes
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import shapiro, levene, bartlett, kstest, boxcox

## Funções

In [ ]:
def analisar_distribuicao_e_boxplot(df, feature, target, figsize_dist=(20, 6), figsize_box=(20, 6), x_rotation=0):
    """
    Gera o gráfico de distribuição, o boxplot e um gráfico de linha para uma característica específica e sua relação com uma variável de destino.
    Destaca em laranja os dois boxes com maior dispersão e exibe essas categorias no console.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os dados.
        feature (str): Nome da coluna da característica a ser analisada.
        target (str): Nome da variável de destino para o boxplot.
        figsize_dist (tuple): Tamanho da figura para o gráfico de distribuição (default: (20, 6)).
        figsize_box (tuple): Tamanho da figura para o boxplot (default: (20, 6)).

    Retorno:
        None: Exibe os gráficos diretamente.
    """
    # Gráfico de distribuição
    plt.figure(figsize=figsize_dist)
    if df[feature].dtype == 'object':  # Variáveis categóricas
        sns.countplot(data=df, x=feature, order=df[feature].value_counts().index)
        plt.title(f"Distribuição de {feature}", fontsize=14)
        plt.xlabel(feature, fontsize=12)
        plt.ylabel("Frequência", fontsize=12)
    else:  # Variáveis numéricas
        sns.histplot(df[feature], kde=True, bins=20, color='green')
        plt.title(f"Distribuição de {feature}", fontsize=14)
        plt.xlabel(feature, fontsize=12)
        plt.ylabel("Frequência", fontsize=12)
    plt.xticks(rotation=x_rotation)  # Inclinar rótulos do eixo x
    plt.grid(True)
    plt.show()

    # Calcular dispersões
    dispersoes = df.groupby(feature)[target].apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
    maiores_dispersoes = dispersoes.nlargest(2)

    # Printar as features com maior dispersão
    print("Features com maior dispersão:")
    for idx, dispersao in maiores_dispersoes.items():
        print(f"- {idx}: {dispersao:.2f}")

    # Criar uma coluna auxiliar para destacar as categorias com maior dispersão
    df['destaque_dispersao'] = df[feature].apply(lambda x: 'Destaque' if x in maiores_dispersoes.index else 'Normal')

    # Gráfico de linha: feature x target
    plt.figure(figsize=(20, 6))
    if df[feature].dtype == 'object':  # Variáveis categóricas
        line_data = df.groupby(feature)[target].mean().reset_index()
        sns.lineplot(data=line_data, x=feature, y=target, marker='o')
    else:  # Variáveis numéricas
        line_data = df[[feature, target]].sort_values(by=feature)
        sns.lineplot(data=line_data, x=feature, y=target, marker='o')
    plt.title(f"Relação entre {feature} e {target}", fontsize=14)
    plt.xlabel(feature, fontsize=12)
    plt.ylabel(target, fontsize=12)
    plt.xticks(rotation=x_rotation)  # Inclinar rótulos do eixo x
    plt.grid(True)
    plt.show()

    # Boxplot com destaque para as maiores dispersões
    plt.figure(figsize=figsize_box)
    sns.boxplot(
        data=df, x=feature, y=target, 
        order=sorted(df[feature].unique()),  # Ordena alfabeticamente as categorias
        hue='destaque_dispersao', 
        palette={"Destaque": "orange", "Normal": "blue"}
    )
    plt.title(f"Boxplot de {target} por {feature} (maiores dispersões destacadas)", fontsize=14)
    plt.xlabel(feature, fontsize=12)
    plt.ylabel(target, fontsize=12)
    plt.xticks(rotation=x_rotation)  # Inclinar rótulos do eixo x
    plt.legend(title="Destaque", loc="upper right")
    plt.grid(True)
    plt.show()

    # Remover a coluna auxiliar
    df.drop(columns='destaque_dispersao', inplace=True)


In [ ]:
def calcular_media_por_grupo_com_quantidade(df, feature, target):
    """
    Calcula a média do target por grupo da feature e adiciona a quantidade de registros por grupo.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os dados.
        feature (str): Nome da feature categórica.
        target (str): Nome da variável numérica.

    Retorno:
        pd.DataFrame: DataFrame com a média e a quantidade de registros por grupo.
    """
    media_por_grupo = df.groupby(feature)[target].agg(['mean', 'count']).reset_index()
    media_por_grupo.columns = ['Feature Value', 'Mean SalePrice', 'qtde_registros']
    media_por_grupo['Feature'] = feature
    return media_por_grupo

In [ ]:
def calcular_normalidade(df, col_grupo, col_valor):
    """
    Calcula os testes de normalidade Shapiro-Wilk e Kolmogorov-Smirnov para cada grupo em um DataFrame.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo todos os registros.
        col_grupo (str): Nome da coluna categórica para agrupar os dados.
        col_valor (str): Nome da coluna numérica para verificar a normalidade.

    Retorno:
        pd.DataFrame: DataFrame com os resultados dos testes de normalidade.
    """
    # Lista para armazenar os resultados
    normalidade_resultados = []

    # Agrupar os dados pelo valor da coluna categórica
    grupos = df.groupby(col_grupo)

    # Iterar sobre os grupos
    for grupo, dados in grupos:
        qtde_registros = len(dados[col_valor])  # Quantidade de registros no grupo

        if qtde_registros >= 3:  # Apenas realizar os testes se houver pelo menos 3 registros
            # Teste Shapiro-Wilk
            stat_sw, p_value_sw = shapiro(dados[col_valor])

            # Teste Kolmogorov-Smirnov
            stat_ks, p_value_ks = kstest(
                (dados[col_valor] - dados[col_valor].mean()) / dados[col_valor].std(),
                cdf='norm'
            )

            # Adicionar resultados ao DataFrame
            normalidade_resultados.append({
                'Grupo': grupo,
                'Shapiro-Wilk p-value': p_value_sw,
                'Kolmogorov-Smirnov p-value': p_value_ks,
                'Normal Distribution (Shapiro-Wilk)': 'Sim' if p_value_sw >= 0.05 else 'Não',
                'Normal Distribution (K-S)': 'Sim' if p_value_ks >= 0.05 else 'Não',
                'Quantidade de Registros': qtde_registros
            })
        else:
            # Caso o grupo não tenha dados suficientes para os testes
            normalidade_resultados.append({
                'Grupo': grupo,
                'Shapiro-Wilk p-value': None,
                'Kolmogorov-Smirnov p-value': None,
                'Normal Distribution (Shapiro-Wilk)': 'Amostra insuficiente',
                'Normal Distribution (K-S)': 'Amostra insuficiente',
                'Quantidade de Registros': qtde_registros
            })

    # Criar e retornar o DataFrame com os resultados
    return pd.DataFrame(normalidade_resultados)

## 1) Leitura do CSV

In [ ]:
# Caminho do arquivo CSV no subdiretório 'dados'
file_path = './dados/AmesHousing.csv'

# Leitura do arquivo CSV
df_AmesHousing = pd.read_csv(file_path)

df_AmesHousing.head()

In [ ]:
df_AmesHousing.shape

In [ ]:
# Selecionar as colunas necessárias
df_AmesHousing = df_AmesHousing[['SalePrice', 'Bedroom AbvGr', 'Garage Cars', 'Neighborhood']]

# Renomear colunas para facilitar a manipulação
df_AmesHousing.columns = ['SalePrice', 'Bedrooms', 'Garage', 'Neighborhood']

In [ ]:
df_AmesHousing.head()

In [ ]:
df_AmesHousing.columns

# 2) Análise Exploratória

**2.1) Bedrooms**

- **Média**: 2,85 quartos;
- **Mediana**: 3 quartos, muito próxima da média, sugerindo uma distribuição aproximadamente simétrica para a variável `Bedrooms`;
- A distribuição está concentrada principalmente entre **2 e 3 quartos**, conforme observado no gráfico de distribuição;
- Há **8 imóveis** com **0 quartos**, o que pode representar outliers ou espaços não configurados como quartos tradicionais (ex.: estúdios ou imóveis comerciais);
- No **gráfico "Relação entre Bedrooms e SalePrice"**, percebe-se que o preço de venda tende a aumentar conforme o número de quartos. Contudo, há uma grande variabilidade de preços dentro de cada número de quartos, o que indica que fatores adicionais (como localização, tamanho geral do imóvel e características internas) influenciam mais significativamente no preço de venda do que apenas o número de quartos;
- No **gráfico "Boxplot do Preço de Venda por Número de Quartos"**, observa-se uma quantidade significativa de outliers especialmente em imóveis com **4 ou mais quartos**. Isso reforça a ideia de que os preços para imóveis maiores podem variar amplamente, possivelmente devido a características exclusivas;
- Apesar de haver algumas amostras de imóveis com **7 ou mais quartos**, o pequeno número de observações limita a análise da dispersão ou tendências consistentes para essa categoria.



**2.2) Número de garagens**
- Média de 1,7668 garagens;
- Maioria das casas possui de 1 a 2 vagas na garagem;
- No gráfico de comparação do número de vagas e o preço, **gráfico "Relação entre Garage e SalePrice"**, observa-se uma influência positiva do número de vagas no preço até 3 vagas, acima de 3 vagas há poucos imóveis;
- No **gráfico "Boxplot do Preço de Venda por Número de Vagas na Garagem"**, observa-se uma maior dispersão em 3 vagas, mas nesse caso, há uma menor frequência de imóveis.

**2.3) Neighborhood**
- **Variável categórica**, no total de 28 bairros distintos;
- Não há nenhum imóvel sem bairro;
- No **gráfico de "Distribuição de Neighbouhood"** é possível obsevar que os bairros possum frequências bastante variadas;
- Na comparação dos bairros com o preço, **gráfico "Relação entre Neighborhood e SalePrice"**, observa-se a influência do bairro nos preços. Alguns apresentam alta dispersão, como o StoneBr, e outros mais homogêneos como o NPkVill;
- Os preços mais altos, observado pelo valor da media no gráfica de Relação entre Neighborhood e SalePrice", são NridgHT e StoneBr, e os com preços mais baixos são IDOTRR e MeadowV. 

In [ ]:
# Resumo estatístico das variáveis numéricas
print("\nResumo estatístico das variáveis:")
print(df_AmesHousing.describe())

# Verificar valores nulos
print("\nValores nulos por coluna:")
print(df_AmesHousing.isnull().sum())

In [ ]:
# Obter os valores distintos da coluna 'Neighborhood'
distinct_neighborhoods = df_AmesHousing['Neighborhood'].unique()
print("Bairros distintos:")
print(distinct_neighborhoods)

# Contar a quantidade de bairros distintos
num_neighborhoods = df_AmesHousing['Neighborhood'].nunique()
print(f"Quantidade de bairros distintos: {num_neighborhoods}")


In [ ]:
# Tratamento dos dados

# Remove a linha nula existente
df_AmesHousing = df_AmesHousing.dropna()

df_AmesHousing.shape

In [ ]:
analisar_distribuicao_e_boxplot(df_AmesHousing, 'Bedrooms' , 'SalePrice')

In [ ]:
# Contar a quantidade de imóveis com 0 quartos
imoveis_com_0_quartos = df_AmesHousing[df_AmesHousing['Bedrooms'] == 0].shape[0]

print(f"Quantidade de imóveis com 0 quartos: {imoveis_com_0_quartos}")


In [ ]:
analisar_distribuicao_e_boxplot(df_AmesHousing, 'Garage' , 'SalePrice')

In [ ]:
analisar_distribuicao_e_boxplot(df_AmesHousing, 'Neighborhood' , 'SalePrice', x_rotation=45)

# 2) Análise estatística ANOVA

A **Análise de Variância (ANOVA)** será utilizada para investigar se as médias de variáveis relacionadas ao preço de venda de imóveis (**`SalePrice`**) são significativamente diferentes em relação a determinadas categorias no dataset **Ames Housing**. O ANOVA avalia a variância **entre os grupos** e a variância **dentro dos grupos** para determinar se as diferenças observadas entre as médias são estatisticamente significativas.

## Hipóteses do Teste ANOVA

O teste ANOVA trabalha com as seguintes hipóteses:

- **Hipótese Nula (\(H_0\))**: As médias de **`SalePrice`** são iguais entre os grupos definidos pelas variáveis categóricas analisadas.
  $$
  H_0 : \mu_1 = \mu_2 = \mu_3 = \dots = \mu_n
  $$

- **Hipótese Alternativa (\(H_A\))**: Pelo menos um grupo tem média de **`SalePrice`** diferente.
  $$
  H_A : \exists \, i, j \, | \, \mu_i \neq \mu_j \, (i \neq j)
  $$

## Variáveis Selecionadas

As variáveis do dataset **Ames Housing** consideradas para a análise são:

- **`SalePrice`** (variável dependente): Representa o preço de venda dos imóveis.
- **`Bedroom AbvGr`** (variável independente): Número de quartos acima do nível do solo.
- **`Garage Cars`** (variável independente): Capacidade da garagem em termos de número de carros.
- **`Neighborhood`** (variável independente): Bairro onde o imóvel está localizado.

## Suposições do ANOVA

Para realizar o ANOVA corretamente, é necessário verificar as seguintes condições:

1. **Independência das Observações**:
   - Os preços de venda dos imóveis (**`SalePrice`**) devem ser independentes para cada nível das variáveis categóricas. No contexto do **Ames Housing**, pode-se assumir que os registros de diferentes propriedades são independentes.

2. **Normalidade**:
   - A distribuição de **`SalePrice`** para cada nível das variáveis categóricas (**`Bedroom AbvGr`**, **`Garage Cars`**, e **`Neighborhood`**) deve seguir uma distribuição normal.
   - Para verificar essa suposição, utilizaremos:
     - **Teste de Shapiro-Wilk**:
       - \( p >= 0.05 \): Distribuição normal (não rejeitamos a hipótese nula).
       - \( p < 0.05 \): Distribuição não normal (rejeitamos a hipótese nula).
     - **Teste de Kolmogorov-Smirnov**:
       - \( p >= 0.05 \): Distribuição normal (não rejeitamos a hipótese nula).
       - \( p < 0.05 \): Distribuição não normal (rejeitamos a hipótese nula).

3. **Homogeneidade de Variâncias**:
   - As variâncias de **`SalePrice`** entre os grupos devem ser iguais (homocedasticidade).
   - Essa condição pode ser avaliada com:
     - **Teste de Levene**: Sensível a desvios de homogeneidade.
     - **Teste de Bartlett**: Verifica igualdade das variâncias assumindo normalidade.

## Objetivo da Análise

O objetivo do teste ANOVA neste dataset é verificar se:

1. O número de quartos (**`Bedroom AbvGr`**) impacta significativamente o preço de venda (**`SalePrice`**).
2. A capacidade da garagem (**`Garage Cars`**) influencia o preço de venda (**`SalePrice`**).
3. Os preços de venda (**`SalePrice`**) variam significativamente entre os bairros (**`Neighborhood`**).

## Etapas do Procedimento

1. **Preparação dos Dados**:
   - Dividir o dataset em grupos com base nos valores únicos de cada variável categórica (**`Bedroom AbvGr`**, **`Garage Cars`**, e **`Neighborhood`**).

2. **Verificação das Suposições**:
   - **Independência**: Confirmar que os registros no dataset são independentes.
   - **Normalidade**: Testar a normalidade de **`SalePrice`** para cada grupo.
   - **Homogeneidade de Variâncias**: Confirmar se as variâncias de **`SalePrice`** são homogêneas entre os grupos.

3. **Execução do Teste ANOVA**:
   - Aplicar o teste ANOVA para verificar se as diferenças nas médias de **`SalePrice`** entre os grupos são significativas.

4. **Interpretação dos Resultados**:
   - Se o valor-p do teste ANOVA for menor que 0.05 (\(p < 0.05\)), rejeitamos \(H_0\) e concluímos que existe uma diferença significativa entre os grupos.
   - Caso contrário (\(p \geq 0.05\)), não há evidências estatísticas para rejeitar \(H_0\), e as médias podem ser consideradas iguais.

---

Este procedimento permitirá avaliar o impacto de características como o número de quartos, a capacidade da garagem e a localização no preço de venda dos imóveis no dataset **Ames Housing**.


In [ ]:
# Aplicar para cada feature
df_bedrooms = calcular_media_por_grupo_com_quantidade(df_AmesHousing, 'Bedrooms', 'SalePrice')
df_garage = calcular_media_por_grupo_com_quantidade(df_AmesHousing, 'Garage', 'SalePrice')
df_neighborhood = calcular_media_por_grupo_com_quantidade(df_AmesHousing, 'Neighborhood', 'SalePrice')

# Combinar os resultados em um único DataFrame
df_neighborhood_media = pd.concat([df_bedrooms, df_garage, df_neighborhood], ignore_index=True)

# Filtrar o DataFrame para grupos com menos de 4 registros
df_filtrado = df_neighborhood_media[df_neighborhood_media['qtde_registros'] < 4]

# Exibir o DataFrame filtrado
df_filtrado.head()


In [ ]:
# Ajusta colunas que contenham no grupo menos de 4 registros, para isso grupos serão agrupados

# Criar uma nova coluna 'Bedrooms_2' com as condições especificadas
df_AmesHousing['Bedrooms_2'] = df_AmesHousing['Bedrooms'].apply(
    lambda x: '2A' if x <= 2 else ('4A' if x >= 4 else x)
)

# Criar uma nova coluna 'Garage_2' com as condições especificadas
df_AmesHousing['Garage_2'] = df_AmesHousing['Garage'].apply(
    lambda x: '1A' if x <= 1 else ('3A' if x >= 3 else x)
)

# Criar a coluna 'Neighborhood_2' com base nos valores de 'Neighborhood'
df_AmesHousing['Neighborhood_2'] = df_AmesHousing['Neighborhood'].apply(
    lambda x: 'Grp_LGGBN' if x in ['Landmrk', 'GrnHill', 'Greens', 'Blueste', 'NPkVill'] else x
)


In [ ]:
# Aplicar para cada feature
df_bedrooms = calcular_media_por_grupo_com_quantidade(df_AmesHousing, 'Bedrooms_2', 'SalePrice')
df_garage = calcular_media_por_grupo_com_quantidade(df_AmesHousing, 'Garage_2', 'SalePrice')
df_neighborhood = calcular_media_por_grupo_com_quantidade(df_AmesHousing, 'Neighborhood_2', 'SalePrice')

# Combinar os resultados em um único DataFrame
df_neighborhood_media = pd.concat([df_bedrooms, df_garage, df_neighborhood], ignore_index=True)

# Filtrar o DataFrame para grupos com menos de 4 registros
df_filtrado = df_neighborhood_media[df_neighborhood_media['qtde_registros'] < 4]

# Exibir o DataFrame filtrado
df_filtrado.head()

In [ ]:
# Exibir o DataFrame consolidado
df_neighborhood_media.head(30)

In [ ]:
# Avalia a normalidade de Bedrooms

In [ ]:
df_resultados = calcular_normalidade(df_AmesHousing, col_grupo='Bedrooms_2', col_valor='SalePrice')

df_normalidade_nao = df_resultados[df_resultados['Normal Distribution (K-S)'] != 'Sim']
df_normalidade_nao.head()

In [ ]:
# Criar novas colunas no DataFrame com as transformações logarítmica e de raiz quadrada
df_AmesHousing['SalePrice_log'] = df_AmesHousing['SalePrice'].apply(lambda x: np.log(x) if x > 0 else None)
df_AmesHousing['SalePrice_sqrt'] = df_AmesHousing['SalePrice'].apply(lambda x: np.sqrt(x) if x >= 0 else None)

In [ ]:
df_resultados = calcular_normalidade(df_AmesHousing, col_grupo='Bedrooms', col_valor='SalePrice')

df_normalidade_nao = df_resultados[df_resultados['Normal Distribution (K-S)'] != 'Sim']
df_normalidade_nao.head(100)

In [ ]:
import pandas as pd
from scipy.stats import shapiro

# Criar um DataFrame para armazenar os resultados do teste de normalidade
normalidade_resultados = []

# Definir as features categóricas a serem analisadas
features = ['Neighborhood', 'Bedrooms', 'Garage']  # Substituir pelos nomes das colunas reais

# Iterar sobre as features e testar normalidade em cada grupo
for feature in features:
    grupos = df_AmesHousing.groupby(feature)
    for grupo, dados in grupos:
        qtde_registros = len(dados['SalePrice'])  # Quantidade de registros no grupo
        if qtde_registros >= 3:  # Garantir que o grupo tem dados suficientes
            stat, p_value = shapiro(dados['SalePrice'])
            normalidade_resultados.append({
                'Feature': feature,
                'Feature Value': grupo,
                'Shapiro-Wilk p-value': p_value,
                'Quantidade de Registros': qtde_registros,
                'Normal Distribution': 'Sim' if p_value >= 0.05 else 'Não'
                
            })
        else:
            # Caso o grupo não tenha dados suficientes para o teste
            normalidade_resultados.append({
                'Feature': feature,
                'Feature Value': grupo,
                'Shapiro-Wilk p-value': None,
                'Quantidade de Registros': qtde_registros,
                'Normal Distribution': 'Amostra insuficiente'
            })

# Criar o DataFrame com os resultados
df_normalidade = pd.DataFrame(normalidade_resultados)

In [ ]:
# Filtrar os resultados onde a distribuição não é normal
df_normalidade_nao = df_normalidade[df_normalidade['Normal Distribution'] == 'Não']

# Exibir os resultados
print("Grupos que não seguem a normalidade:")
df_normalidade_nao.head()